In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os
from PIL import Image

# Load the saved model
model = models.resnet18(weights='ResNet18_Weights.DEFAULT')
model.fc = nn.Linear(model.fc.in_features, 1000)  # Adjust to match the original model's output units
model.load_state_dict(torch.load(r"C:\Users\Admin\PythonScripts\RohitPSACM\pepe_non_pepe_classifier.pth"))
model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [24]:
image_path = r"C:\Users\Admin\PythonScripts\RohitPSACM\test\not_pepe\isNotPepe96.png"  # Replace with the path to your image
image = Image.open(image_path)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)

In [29]:
#define transforms
#define data augmentations

dataTransforms = {
    "train": transforms.Compose([
      transforms.Resize(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    "val": transforms.Compose([
      transforms.Resize(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),


    "test" : transforms.Compose([
      transforms.Resize(224),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [30]:
folder_path = r"C:\Users\Admin\PythonScripts\RohitPSACM"

imgDatasets = {x: datasets.ImageFolder(os.path.join(folder_path, x), dataTransforms[x]) for x in ['train', 'val', 'test'] }

dataloaders = {x: torch.utils.data.DataLoader(imgDatasets[x], batch_size=4, shuffle=True, num_workers=2) for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(imgDatasets[x]) for x in ['train', 'val', 'test']}
print(dataset_sizes)

class_names = imgDatasets['test'].classes
class_names

{'train': 805, 'val': 383, 'test': 180}


['Pepe', 'not_pepe']

In [31]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Use all parameters
# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [32]:
model = models.resnet18(weights='ResNet18_Weights.DEFAULT')
model.fc = nn.Linear(model.fc.in_features, 1000)  # Adjust to match the original model's output units
model.load_state_dict(torch.load(r"C:\Users\Admin\PythonScripts\RohitPSACM\flower_classification_model0.pth"))
model.eval()

# Create a new model with the correct final layer
new_model = models.resnet18(weights='ResNet18_Weights.DEFAULT')
new_model.fc = nn.Linear(new_model.fc.in_features, 2)  # Adjust to match the desired output units

# Copy the weights and biases from the loaded model to the new model
new_model.fc.weight.data = model.fc.weight.data[0:2]  # Copy only the first 2 output units
new_model.fc.bias.data = model.fc.bias.data[0:2]

In [33]:
num_epochs = 10
for epoch in range(num_epochs):
    for phase in ['train', 'val', 'test']:
        if phase == 'train':
            new_model.train()
        else:
            new_model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training complete!")

train Loss: 0.0280 Acc: 0.9925
val Loss: 0.0253 Acc: 0.9948
test Loss: 0.0549 Acc: 0.9833
train Loss: 0.0313 Acc: 0.9888
val Loss: 0.0213 Acc: 0.9974
test Loss: 0.0627 Acc: 0.9722
train Loss: 0.0287 Acc: 0.9888
val Loss: 0.0307 Acc: 0.9948
test Loss: 0.0620 Acc: 0.9778
train Loss: 0.0243 Acc: 0.9938
val Loss: 0.0319 Acc: 0.9948
test Loss: 0.0667 Acc: 0.9778
train Loss: 0.0289 Acc: 0.9901
val Loss: 0.0270 Acc: 0.9948
test Loss: 0.0458 Acc: 0.9833
train Loss: 0.0241 Acc: 0.9913
val Loss: 0.0305 Acc: 0.9948
test Loss: 0.0578 Acc: 0.9833
train Loss: 0.0312 Acc: 0.9913
val Loss: 0.0295 Acc: 0.9948
test Loss: 0.0643 Acc: 0.9722
train Loss: 0.0251 Acc: 0.9925
val Loss: 0.0322 Acc: 0.9948
test Loss: 0.0671 Acc: 0.9778
train Loss: 0.0307 Acc: 0.9913
val Loss: 0.0279 Acc: 0.9948
test Loss: 0.0669 Acc: 0.9722
train Loss: 0.0242 Acc: 0.9913
val Loss: 0.0213 Acc: 0.9974
test Loss: 0.0694 Acc: 0.9722
Training complete!


In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os


trainLoss = dict([])
trainAcc = dict([])
valLoss = dict([])
valAcc = dict([])
testLoss = dict([])
testAcc = dict([])

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Set data directories
data_dir = folder_path
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# Load datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

# Define model architecture (using pre-trained ResNet18)
model = models.resnet18(weights = None)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: Pepe and Non-Pepe
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

#num_epochs = 15

# Training function
def train_model(model, criterion, optimizer, num_epochs = 15):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val', 'test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if phase == 'train':
                trainLoss[epoch + 1] = f"{epoch_loss:.4f}"
                trainAcc[epoch + 1] = f"{epoch_acc:.4f}"

            elif phase == 'val':
                valLoss[epoch + 1] = f"{epoch_loss:.4f}"
                valAcc[epoch + 1] = f"{epoch_acc:.4f}"

            elif phase == 'test':
                testLoss[epoch + 1] = f"{epoch_loss:.4f}"
                testAcc[epoch + 1] = f"{epoch_acc:.4f}"


            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

# Train the model
train_model(model, criterion, optimizer, num_epochs=15)

# Save the trained model
torch.save(model.state_dict(), 'pepe_non_pepe_classifier.pth')

print("Train Losses: ", trainLoss)
print("Train Accuracy: ", trainAcc)
print("Validation Losses: ", valLoss)
print("Validation Accuracy: ", valAcc)
print("Testing Losses: ", testLoss)
print("Testing Accuracy: ", testAcc)


Epoch 1/15
----------
train Loss: 0.4945 Acc: 0.7516
val Loss: 0.3272 Acc: 0.8825
test Loss: 0.3666 Acc: 0.8333
Epoch 2/15
----------
train Loss: 0.4665 Acc: 0.7752
val Loss: 0.2929 Acc: 0.9008
test Loss: 0.3456 Acc: 0.8722
Epoch 3/15
----------
train Loss: 0.4279 Acc: 0.8050
val Loss: 0.3256 Acc: 0.8642
test Loss: 0.4148 Acc: 0.8611
Epoch 4/15
----------
train Loss: 0.4787 Acc: 0.7615
val Loss: 0.3704 Acc: 0.8825
test Loss: 0.4493 Acc: 0.8500
Epoch 5/15
----------
train Loss: 0.4157 Acc: 0.8124
val Loss: 0.3135 Acc: 0.9008
test Loss: 0.3902 Acc: 0.8389
Epoch 6/15
----------
train Loss: 0.4244 Acc: 0.8112
val Loss: 0.3150 Acc: 0.8877
test Loss: 0.4028 Acc: 0.8333
Epoch 7/15
----------
train Loss: 0.4009 Acc: 0.8112
val Loss: 0.2875 Acc: 0.8956
test Loss: 0.3786 Acc: 0.8333
Epoch 8/15
----------
train Loss: 0.3819 Acc: 0.8311
val Loss: 0.2839 Acc: 0.8982
test Loss: 0.3690 Acc: 0.8667
Epoch 9/15
----------
train Loss: 0.3850 Acc: 0.8335
val Loss: 0.2930 Acc: 0.8903
test Loss: 0.3841 Acc:

In [27]:
torch.save(model.state_dict(), 'pepe_non_pepe_classifier.pth')
print(trainLoss)

{1: 0.4945, 2: 0.4665, 3: 0.4279, 4: 0.4787, 5: 0.4157, 6: 0.4244, 7: 0.4009, 8: 0.3819, 9: 0.385, 10: 0.3708, 11: 0.3832, 12: 0.3636, 13: 0.3432, 14: 0.3537, 15: 0.3406}
